In [391]:
def scoring(s):
    
    score = 0
    
    for word in solutions:
        if word[0] == s[0]:
            score += 2
        elif word[0] in s:
            score +=1
        if word[1] == s[1]:
            score += 2
        elif word[1] in s:
            score +=1
        if word[2] == s[2]:
            score += 2
        elif word[2] in s:
            score +=1
        if word[3] == s[3]:
            score += 2
        elif word[3] in s:
            score +=1
        if word[4] == s[4]:
            score += 2
        elif word[4] in s:
            score +=1
            
    return score/len(solutions)

In [190]:
import pickle
import random

import pandas as pd


global init

init_flag = False


def init():
    global exs, letters, solutions, test, actual_solution, attempts, df
    
    attempts = []
    
    with open('solutions.pkl', 'rb') as f:
        solutions = pickle.load(f)
        
    with open('test_set.pkl', 'rb') as f:
        test = pickle.load(f)
        
    actual_solution = random.choice(solutions)
#     actual_solution = 'aside'
        
#     df = pd.read_csv('scored_words.csv')
    df = pd.read_csv('scored_solution.csv')
    
    alphabet = [s for s in 'abcdefghijklmnopqrstuvwxyz']
    
    letters = {
        '1': alphabet,
        '2': alphabet,
        '3': alphabet,
        '4': alphabet,
        '5': alphabet
    }
    
    exs = {
        '1': [],
        '2': [],
        '3': [],
        '4': [],
        '5': []
    }

        
def guess(count, verbose):
    global attempts, df
    
    yellows, exemptions = [], []
    greens = [None, None, None, None, None]
    
    if count == 0:
        attempt = 'stale'
    
    elif count == 1:
        if len(solution) > 10:
            attempt = 'irony'
        else:
            attempt = df['words'].iloc[0]
            if attempt in attempts:
                df = df[~df['words'].isin([attempt])]
                attempt = df['words'].iloc[0]
    else:
        attempt = df['words'].iloc[0]
        if attempt in attempts:
            df = df[~df['words'].isin([attempt])]
            attempt = df['words'].iloc[0]
            
    attempts.append(attempt)
    
    if verbose:
        print('Guesses: ' + str(attempts))
        print('Solution: ' + actual_solution)
        
    for i, character in enumerate(attempt):
        if character == actual_solution[i]:
            greens[i] = character
        elif character in actual_solution:
            yellows.append(character)
            exs[str(i+1)] = character
        else:
            exemptions.append(character)
            
    return greens, yellows, exemptions, attempt


def iteration(verbose=False):
    global df, init_flag, attempts, solution
    
    stop = False
    count = 0
    while stop == False:

        if init_flag == False:
            init()
            init_flag = True

        greens, yellows, exemptions, attempt = guess(count, verbose)

        for i, green in enumerate(greens):
            if green:
                letters[str(i+1)] = green

        solution = [word for word in solutions if all(x in word for x in yellows) 
                     and any(x in word[0] for x in letters['1'])
                     and any(x in word[1] for x in letters['2'])
                     and any(x in word[2] for x in letters['3'])
                     and any(x in word[3] for x in letters['4'])
                     and any(x in word[4] for x in letters['5'])
                     and not any(x in word[0] for x in exs['1'] if exs['1'])
                     and not any(x in word[1] for x in exs['2'] if exs['2'])
                     and not any(x in word[2] for x in exs['3'] if exs['3'])
                     and not any(x in word[3] for x in exs['4'] if exs['4'])
                     and not any(x in word[4] for x in exs['5'] if exs['5'])
                     and not any(x in word for x in exemptions)
                    ]
        
        if verbose:
            print('Solution list: ' + str(solution))
            
        df = df[df['words'].isin(solution)].sort_values('scoring', ascending=False)
        count += 1
        if len(solution) == 1:
            if attempt not in solution:
                count += 1
            stop = True
            init_flag = False
        if count == 6:
#             print('failed')
#             print(actual_solution)
            stop = True
            init_flag = False
    return count

In [ ]:
count = 0

for i in range(1000):
    count += iteration()
    
count/1000